In [1]:
from __future__ import print_function
import os
import sys
import math
import pickle
import boto3
import os
import numpy as np
import pandas as pd
# from tqdm import tqdm
import time
import argparse
import logging
import re

# tqdm.pandas()
# pandarallel.initialize(progress_bar=True)
# bucket = os.environ.get("BUCKET_NAME", " ")
# raw_data_folder = os.environ.get("RAW_DATA", " ")

s3client = boto3.client('s3')

In [3]:
########################################
# 从s3同步数据
########################################
s3client = boto3.client('s3')


def sync_s3(file_name_list, s3_folder, local_folder):
    for f in file_name_list:
        print("file preparation: download src key {} to dst key {}".format(os.path.join(
            s3_folder, f), os.path.join(local_folder, f)))
        s3client.download_file(bucket, os.path.join(
            s3_folder, f), os.path.join(local_folder, f))


def write_to_s3(filename, bucket, key):
    print("upload s3://{}/{}".format(bucket, key))
    with open(filename, 'rb') as f:  # Read in binary mode
        # return s3client.upload_fileobj(f, bucket, key)
        return s3client.put_object(
            ACL='bucket-owner-full-control',
            Bucket=bucket,
            Key=key,
            Body=f
        )

def write_str_to_s3(content, bucket, key):
    print("write s3://{}/{}, content={}".format(bucket, key, content))
    s3client.put_object(Body=str(content).encode("utf8"), Bucket=bucket, Key=key, ACL='bucket-owner-full-control')

default_bucket = 'aws-gcr-rs-sol-demo-ap-southeast-1-522244679887'
default_prefix = 'sample-data'
parser = argparse.ArgumentParser()
parser.add_argument('--bucket', type=str, default=default_bucket)
parser.add_argument('--prefix', type=str, default=default_prefix)
args, _ = parser.parse_known_args()
bucket = args.bucket
prefix = args.prefix

print("bucket={}".format(bucket))
print("prefix='{}'".format(prefix))

out_s3_path = "s3://{}/{}/feature/content/inverted-list".format(bucket, prefix)

local_folder = 'info'
if not os.path.exists(local_folder):
    os.makedirs(local_folder)
# 行为/物品数据同步
file_name_list = ['action.csv']
s3_folder = '{}/system/action-data'.format(prefix)
sync_s3(file_name_list, s3_folder, local_folder)
file_name_list = ['item.csv']
s3_folder = '{}/system/item-data'.format(prefix)
sync_s3(file_name_list, s3_folder, local_folder)
file_name_list = ['news_id_news_feature_dict.pickle']
s3_folder = '{}/feature/content/inverted-list'.format(prefix)
sync_s3(file_name_list, s3_folder, local_folder)

bucket=aws-gcr-rs-sol-demo-ap-southeast-1-522244679887
prefix='sample-data'
file preparation: download src key sample-data/system/action-data/action.csv to dst key info/action.csv
file preparation: download src key sample-data/system/item-data/item.csv to dst key info/item.csv
file preparation: download src key sample-data/feature/content/inverted-list/news_id_news_feature_dict.pickle to dst key info/news_id_news_feature_dict.pickle


In [14]:
file_to_load = open("info/news_id_news_feature_dict.pickle", "rb")
news_id_news_feature_dict = pickle.load(file_to_load)

In [4]:
df_filter_item = pd.read_csv('info/item.csv',sep='_!_',names=['news_id','type_code','type','title','keywords','popularity','new'])

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [5]:
df_filter_item.head()

,news_id,type_code,type,title,keywords,popularity,new
0,6423135627981619458,115,news_agriculture,为什么现在农村人很少到广州等地打工反而转战江苏浙江,"一窝蜂,浙江,农村,江苏,广州",0,0
1,6481778451320668430,106,news_house,河北衡水人天津集体户口在天津买房合适还是回衡水买呢,"集体户口,天津,衡水,房价,大都市",3,0
2,6487781016990646541,108,news_edu,襄阳的湖北交通大学将来会不会成为全国五大交校之一,"襄阳,西南交通大学,交通大学,隆中,北京交通大学",9,1
3,6492610374146195725,108,news_edu,如果哈工大不在哈尔滨而是在北京那现在哈工大会不会牛到飞起,"北京,哈工大,哈尔滨",9,0
4,6512940160806551816,109,news_tech,昆山江阴义乌哪个发展最好,"义乌,物联网,昆山,江阴,小商品",7,0


In [6]:
len(df_filter_item)

2660

In [7]:
df_filter_action = pd.read_csv('info/action.csv',sep='_!_',names=['user_id','news_id','timestamp','action_type','action'])

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [8]:
df_filter_action.head()

,user_id,news_id,timestamp,action_type,action
0,52a23654-9dc3-11eb-a364-acde48001122,6552345461607367172,1618477588,1,0
1,52a23654-9dc3-11eb-a364-acde48001122,6552332581256299016,1618472565,1,0
2,52a23654-9dc3-11eb-a364-acde48001122,6552130363123040771,1618467016,1,0
3,52a23654-9dc3-11eb-a364-acde48001122,6475484594673025293,1618462187,1,1
4,52a238fc-9dc3-11eb-a364-acde48001122,6552277802022863374,1618468013,1,0


In [9]:
df_item_stats = df_filter_action[['news_id','action_type','action']]
df_item_stats = df_item_stats.groupby(['news_id','action_type']).sum()
df_item_stats = df_item_stats.reset_index()
df_item_stats['action'] = df_item_stats['action'] / df_item_stats['action'].abs().max() * 10.0
df_item_stats = df_item_stats.drop([0])
df_item_stats.head()

,news_id,action_type,action
1,6320151429650579970,1,3.333333
2,6418014704991469826,1,1.111111
3,6422082903882072321,1,0.000000
4,6422169653342109954,1,2.222222
5,6423135627981619458,1,2.222222


In [10]:
pd_merge_result = pd.merge(df_filter_item, df_item_stats, on="news_id", how="left").drop(columns=['action_type'])
pd_merge_result = pd_merge_result.fillna(0)
# row_has_NaN = pd_merge_result.isnull().any(axis=1)

In [11]:
pd_drop = pd_merge_result.drop(columns=['popularity']).rename(columns={"action":"popularity"})

In [12]:
pd_drop.loc[pd_drop.new == 1, 'popularity'] = 10.0
pd_drop.loc[pd_drop.new == 1, 'new'] = 0

In [13]:
# generate dict_id_keywords for tfidf
dict_keywords_id = {}
for row in df_filter_item.iterrows():
    item_row = row[1]
    program_id = str(item_row['news_id'])
    for kw in item_row['keywords'].split(','):
        if kw not in dict_keywords_id.keys():
            dict_keywords_id[kw] = [program_id]
            continue
        current_list = dict_keywords_id[kw]
        current_list.append(program_id)
        dict_keywords_id[kw].append(program_id)
n_keyword_whole = len(dict_keywords_id)

In [18]:
def get_tfidf(category_property):
    if not category_property or str(category_property).lower() in ['nan', 'nr', '']:
        return [None]
    if not category_property:
        return [None]
    value = [item.strip() for item in category_property.split(',')]
    keywords_tfidf = {}
    for keyword in value:
        current_score = 1 / len(value)*math.log(n_keyword_whole / len(dict_keywords_id[keyword]))
        keywords_tfidf[keyword] = current_score
    return keywords_tfidf
        
def get_category(category_property):
    if not category_property or str(category_property).lower() in ['nan', 'nr', '']:
        return [None]
    if not category_property:
        return [None]
    return [item.strip().lower() for item in category_property.split(',')]
            
def get_single_item(item):
    if not item or str(item).lower().strip() in ['nan', 'nr', '']:
        return [None]
    return [str(item).lower().strip()]

def get_entities(news_id, news_id_news_feature_dict=news_id_news_feature_dict):
    title_result = []
    title_result.append(news_id_news_feature_dict[str(news_id)]['words'])
    title_result.append(news_id_news_feature_dict[str(news_id)]['entities'])
    return title_result

def single_dict(raw_dict, feat, item_id):
    if feat not in raw_dict.keys():
        raw_dict[feat] = [item_id]
    else:
        current_list = raw_dict[feat]
        current_list.append(item_id)
        raw_dict[feat] = current_list

def list_dict(raw_dict, feat_list, item_id):
    for feat in feat_list:
        single_dict(raw_dict, feat, item_id)

def update_popularity(item_df, action_df):
    pd_merge_result = pd.merge(item_df, action_df, on="news_id", how="left").drop(columns=['action_type'])
    pd_merge_result = pd_merge_result.fillna(0)
    df_update = pd_merge_result.drop(columns=['popularity']).rename(columns={"action":"popularity"})
    df_update.loc[df_update.new == 1, 'popularity'] = 10.0
    df_update.loc[df_update.new == 1, 'new'] = 0
    return df_update
        
def sort_by_score(df):
    logging.info("sort_by_score() enter, df.columns: {}".format(df.columns))
    df['popularity'].fillna(0, inplace=True)

    df['popularity_log'] = np.log1p(df['popularity'])
    popularity_log_max = df['popularity_log'].max()
    popularity_log_min = df['popularity_log'].min()

    df['popularity_scaled'] = ((df['popularity_log'] - popularity_log_min) / (
            popularity_log_max - popularity_log_min)) * 10

    df_sorted = df.sort_values(by='popularity_scaled', ascending=False)
    
    df_sorted = df_sorted.drop(
        ['popularity_log', 'popularity_scaled'], axis=1)

    logging.info("sort_by_score() return, df.columns: {}".format(df_sorted.columns))
    return df_sorted

def get_bucket_key_from_s3_path(s3_path):
    m = re.match(r"s3://(.*?)/(.*)", s3_path)
    return m.group(1), m.group(2)

def gen_pickle_files(df, action_df):
    df_update = update_popularity(df, action_df)
    df_sort = sort_by_score(df_update)
    
    news_id_news_property_dict = {}
    news_type_news_ids_dict = {}
    news_keywords_news_ids_dict = {}
    news_entities_news_ids_dict = {}
    news_words_news_ids_dict = {}
    
    for row in df_sort.iterrows():
        item_row = row[1]
        program_id = str(item_row['news_id'])
        current_entities = get_entities(program_id)[1]
        current_words = get_entities(program_id)[0]
        program_dict = {
            'title': get_single_item(item_row['title']),
            'type': get_single_item(item_row['type']),
            'keywords': get_category(item_row['keywords']),
            'tfidf': get_tfidf(item_row['keywords']),
            'entities': current_entities,
            'words': current_words
        }
        news_id_news_property_dict[program_id] = program_dict
        list_dict(news_type_news_ids_dict, program_dict['type'], program_id)
        list_dict(news_keywords_news_ids_dict, program_dict['keywords'], program_id)
        list_dict(news_entities_news_ids_dict, program_dict['entities'], program_id)
        list_dict(news_words_news_ids_dict, program_dict['words'], program_id)

    result_dict = {
        'news_id_news_property_dict': news_id_news_property_dict,
        'news_type_news_ids_dict': news_type_news_ids_dict,
        'news_keywords_news_ids_dict': news_keywords_news_ids_dict,
        'news_entities_news_ids_dict': news_entities_news_ids_dict,
        'news_words_news_ids_dict': news_words_news_ids_dict
    }
    return result_dict

In [19]:
rd = gen_pickle_files(df_filter_item, df_item_stats)

In [20]:
bucket, out_prefix = get_bucket_key_from_s3_path(out_s3_path)
for dict_name, dict_val in rd.items():
    file_name = f'{dict_name}.pickle'
    # print("pickle =>", file_name)
    out_file = open(file_name, 'wb')
    pickle.dump(dict_val, out_file)
    out_file.close()
    # s3_url = S3Uploader.upload(file_name, out_s3_path)
    s3_url = write_to_s3(file_name, bucket, f'{out_prefix}/{file_name}')
    logging.info("write {}".format(s3_url))

upload s3://aws-gcr-rs-sol-demo-ap-southeast-1-522244679887/sample-data/feature/content/inverted-list/news_id_news_property_dict.pickle
upload s3://aws-gcr-rs-sol-demo-ap-southeast-1-522244679887/sample-data/feature/content/inverted-list/news_type_news_ids_dict.pickle
upload s3://aws-gcr-rs-sol-demo-ap-southeast-1-522244679887/sample-data/feature/content/inverted-list/news_keywords_news_ids_dict.pickle
upload s3://aws-gcr-rs-sol-demo-ap-southeast-1-522244679887/sample-data/feature/content/inverted-list/news_entities_news_ids_dict.pickle
upload s3://aws-gcr-rs-sol-demo-ap-southeast-1-522244679887/sample-data/feature/content/inverted-list/news_words_news_ids_dict.pickle


In [15]:
n = 0
for k, v in news_id_news_feature_dict.items():
    print("k {} v {}".format(k,v))
    if n > 10:
        break
    n = n + 1

k 6423135627981619458 v {'entities': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0], 'words': [1, 2, 0, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 0, 0, 0]}
k 6481778451320668430 v {'entities': [3, 0, 4, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'words': [13, 0, 14, 15, 16, 17, 14, 18, 19, 20, 21, 22, 23, 18, 24, 0]}
k 6487781016990646541 v {'entities': [0, 0, 5, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0], 'words': [25, 26, 27, 28, 29, 30, 31, 32, 31, 33, 34, 35, 36, 37, 0, 38]}
k 6492610374146195725 v {'entities': [0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0], 'words': [39, 40, 32, 17, 41, 42, 43, 17, 44, 45, 2, 40, 31, 32, 31, 0]}
k 6512940160806551816 v {'entities': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'words': [46, 47, 48, 49, 50, 51, 52, 53, 0, 0, 0, 0, 0, 0, 0, 0]}
k 6520157116831891716 v {'entities': [0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0, 0, 0, 0], 'words': [54, 55, 56, 0, 57, 26, 58, 59, 0, 0, 0, 0, 0, 0, 0, 0]}
k 6520548609547567367 v {'entities': [0, 0, 0, 0, 0, 0, 0, 0, 0